In [108]:
import os
from pathlib import Path
import json
import multiprocessing

import cv2
import pandas as pd
import requests
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# Закачаем данные

In [109]:
def func(inp):
    fname, url = inp
    if not url:
        print(fname + ' no url')
        return
    res = sorted([(url.lower().find('.jpg'), 'jpg'),
                 (url.lower().find('.jpeg'), 'jpg'),
                 (url.lower().find('.png'), 'png')])[-1][1]
    fname += res
#     print(fname)
    if os.path.isfile(fname):
        return #continue
    try:
        r = requests.get(url, allow_redirects=True, timeout=5)
        if len(r.content) < 1024:
            return
        with open(fname, 'wb') as f:
            f.write(r.content)
    except:
        pass #print(url + ' failed')

In [121]:
def download_photos(fname, folder, ban_list):
    pool = multiprocessing.Pool(8)

    data = pd.read_csv(fname)
    
    if not os.path.isdir(folder):
        os.mkdir(folder)
    urls = []
    for d in tqdm(data.values):
        subfolder, fname = d[1].split('/')
        if subfolder in ban_list:
            continue
        subfolder = folder + subfolder
        if not os.path.isdir(subfolder):
            os.mkdir(subfolder) 
        
        urls.append((d[1], d[2]))

    
    print(len(urls))
    print(urls[:10])
    pool.map(func, urls)

In [123]:
# request ids to be ignored
ban_list = [
        '051394c7-5884-4d38-a18e-9a822954052d',
        '7c58ac38-7d1c-4f19-b112-b5b43f50a6ef',
        '1a7a7474-9855-4677-834a-d7a167b206d1']
folder = '/home/jupyter/work/resources/DaytimeTranslation/data/images/'
fname = '/home/jupyter/work/resources/DaytimeTranslation/data/links.csv'

download_photos(fname, folder, ban_list)


10603
[('11232fa9-4f70-4ede-9cd2-a91b8922f38a/00000.', 'https://www.zastavki.com/pictures/originals/2015/World___Travel_and_Tourism_Himalaya_mountain_peaks_in_Nepal_096364_.jpg'), ('11232fa9-4f70-4ede-9cd2-a91b8922f38a/00001.', 'https://i.artfile.ru/2048x1239_897404_[www.ArtFile.ru].jpg'), ('11232fa9-4f70-4ede-9cd2-a91b8922f38a/00002.', 'https://cdn.pixabay.com/photo/2016/07/19/22/42/sunset-1529136_960_720.jpg'), ('11232fa9-4f70-4ede-9cd2-a91b8922f38a/00003.', 'https://www.sunhome.ru/i/wallpapers/128/zakat-nebo-gori.orig.jpg'), ('11232fa9-4f70-4ede-9cd2-a91b8922f38a/00004.', 'https://img4.goodfon.ru/wallpaper/nbig/7/2b/nebo-trava-solntse-oblaka-luchi-peizazh-zakat-gory-priroda-t.jpg'), ('11232fa9-4f70-4ede-9cd2-a91b8922f38a/00005.', 'https://img4.goodfon.ru/original/2048x1366/4/eb/gory-alpy-vecher-svet-oblaka-nebo.jpg'), ('11232fa9-4f70-4ede-9cd2-a91b8922f38a/00006.', 'https://c.wallhere.com/photos/51/a5/landscape_nature_blue_mist_sunset_forest_mountains_sky-93254.jpg!d'), ('11232fa9-

# Почистим не открывающиеся

In [5]:
data_path = Path('/home/jupyter/work/resources/DaytimeTranslation/data/images')
filenames = list(str(p) for p in data_path.glob('**/*.jpg')) + list(str(p) for p in data_path.glob('**/*.png'))

In [ ]:
len(filenames)

In [94]:
ban_lst = []
for fname in tqdm(filenames):
    try:
        image = cv2.imread(fname)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    except cv2.error:
        ban_lst.append(fname)

In [95]:
len(ban_lst)

70

In [96]:
for fname in tqdm(ban_lst):
    os.remove(fname)